In [1]:
#Installing dependencies
!pip install sentence-splitter
!pip install transformers
!pip install SentencePiece
!pip3 install num2words
!pip3 install pandas


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

data = pd.read_csv("data/GMB_train.csv",encoding='unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [3]:
data["Tag"] = data["Tag"].str.replace("B-", "")

data["Tag"] = data["Tag"].str.replace("I-", "")

#Removing .
data = data[~data["Word"].isin(["."])]
data = data.reset_index(inplace = False,drop=True)
data.to_csv("Interim.csv")

In [4]:
sentence_num = data.at[0,"Sentence #"]
for i in data.index:
    
    if len(str(data.at[i,"Sentence #"])) != 3:
        sentence_num = data.at[i,"Sentence #"]
    else:
        data.at[i,"Sentence #"] = sentence_num

data.to_csv("data_tag_transformed.csv",index=False)

In [5]:
#importing the PEGASUS Transformer model
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
 
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)
 
#setting up the model
def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [6]:
#test input sentence
import time

t1 = time.time()
text = "Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawl of British troops form that country"

#printing response
print ((get_response(text, 5)))

print ("Time-------",(time.time()-t1))

['Thousands of people marched through London to protest the war in Iraq and demand the withdrawal of British troops.', 'Thousands of protesters marched through London to demand the withdrawal of British troops from Iraq.', 'Protesters marched through London to demand the withdrawal of British troops from Iraq.', 'Thousands of demonstrators marched through London to protest the war in Iraq and demand the withdrawal of British troops.', 'Thousands of demonstrators marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country.']
Time------- 0.656456708908081


In [7]:
# data = pd.read_csv("test_interim.csv")

In [8]:
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [9]:
col_list = ["Sentence #","Original_Sentence","T1","T2","T3","T4","T5"]
data2 = pd.DataFrame(columns=col_list)
data2.head()

,Sentence #,Original_Sentence,T1,T2,T3,T4,T5


In [ ]:
import re
time1 = time.time()
c = 0

for i in range(1,47960):
    if (i%500 == 0):
        print (i)
    temp_data = data[data["Sentence #"].isin(["Sentence: "+ "% s" % i])]
    word_list = temp_data["Word"].to_list()
    sentence = " ".join(word_list)
    
    #print (sentence)

    t_list = (get_response(sentence, 5))
    #print (t_list)
    
    temp_data2 = temp_data[~temp_data["Tag"].isin(["O"])]
    temp_data2 = temp_data2.reset_index(inplace = False,drop=True)
    #print(temp_data2)
    
    final_list = []
    
    
    for item in t_list:
        #print (item)
        str1 = ""
        item = list(item.split(" "))
        for term in item:
            term = re.sub(r'[.]', '', term)
            temp_data3 = temp_data2[temp_data2["Word"].isin([term])]
            #print (term)
            temp_data3 = temp_data3.reset_index(inplace = False,drop=True)
            #print (temp_data3)
            if len(temp_data3) != 0:
                #print("Check1")
#                 print(temp_data3)
#                 print (term)
                str1 = str1 + "< " + temp_data3.at[0,"Tag"] + " > " + temp_data3.at[0,"Word"] + " </ " + temp_data3.at[0,"Tag"] + " > "
            else:
                #print ("Check2")
                str1 = str1 + term + " "
        final_list.append(str1)
    
    data2.at[c,"Sentence #"] = "Sentence: "+ "% s" % i
    data2.at[c,"Original_Sentence"] = sentence
    for j in range (1,6):
        #print (c)
        data2.at[c,"T"+ "% s" % j] = final_list[j-1]
    c = c+1
    
time2 = time.time()

In [ ]:
print (final_list)

In [ ]:
print (data2.head())

In [ ]:
data2.to_csv("Pegasus_augmentations.csv")

In [ ]:
print ((time2-time1))